In [1]:
import os, sys, json
sys.path.append("/home/tristan/projects/LLM/AutoCog")
from autocog.language.sta.frontend import frontend
from autocog.utility.pynb import wrap_graphviz, display

## Single prompt and _compact_

**Note**:
 - `text` is implicitly `text<INF>` (meaning no limit on number of tokens) 
   - **but** can only be used if target of a channel

In [2]:
tree = frontend("""
define default_number_of_choices=4;
argument N=$default_number_of_choices;
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[$N] is {
            value is text;
            correct is bool;
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return {
        from .answer;
    }
    annotate {
        .topic           as "the general category from which the question was taken";
        .question        as "the question that you have to answer";
        .choices         as "you judge whether each choice is correct or not";
        .choices.value   as "the value of the choice";
        .choices.correct as "you decide whether this choice is correct or not";
        .answer          as "you repeat the value of the choice that best answer the question";
    }
}
""")
wrap_graphviz(tree.toGraphViz())

## Reusable data-structure

**Note**:
 - showing shorthand for single element `return`

In [3]:
tree = frontend("""
struct a_choice {
    is {
        value   is text;
        correct is bool;
    }
    annotate {
        .value   as "the value of the choice";
        .correct as "you decide whether this choice is correct or not";
    }
}

prompt mmlu_main {
    is {
        topic      is text;
        question   is text;
        choices[4] is a_choice;
        answer     is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
    }
    return .answer;
    annotate {
        .topic    as "the general category from which the question was taken";
        .question as "the question that you have to answer";
        .choices  as "you judge whether each choice is correct or not";
        .answer   as "you repeat the value of the choice that best answer the question";
    }
}
""")
wrap_graphviz(tree.toGraphViz())

## Flow between prompts and formats

**Note**:
 - annotation with f-exp `f"{var}"`
 - global `flow` declares possible entry point
   - entry point defaults to `main`
   - if not precised the first prompt defaults to be `main`

In [4]:
tree = frontend("""
format sentence {
    argument N=30;
    is text<$N>;
    annotate f"A grammatically correct sentence made of at most {N} tokens.";
}

prompt mmlu_main {
    define K=50;
    is {
        topic       is text;
        question    is text;
        hyphothesis is sentence<$K>;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
    }
    flow {
        to mmlu_choose;
    }
}

prompt mmlu_choose {
    is {
        topic       is text;
        question    is text;
        hyphothesis is sentence;
        choices[4]  is text;
        answer      is repeat(.choices.value);
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
        to .hyphothesis   from mmlu_main.hyphothesis;
    }
    return .answer;
}

flow {
    to mmlu_main as "main";
}
""")
wrap_graphviz(tree.toGraphViz())

## Iterations

**Note**:
 - use of `as` in both `flow` and `return`, when prompt ends LLM will have to choose between `retry` and `ready`
   - without `as` choice would have been `mmlu_main` or `return`
 - `[5]` in the flow section limits the trip count on that branch ensuring termination

In [5]:
tree = frontend("""
prompt mmlu_main {
    is {
        topic         is text;
        question      is text;
        choices[4]    is text;
        previous[0:1] is text<20>;
        answer        is repeat(.choices.value);
        reflect       is text<20>;
    }
    channel {
        to .topic         from ?topic;
        to .question      from ?question;
        to .choices.value from ?choices;
        to .previous      from .reflect;
    }
    flow {
        to mmlu_main[5] as "retry";
    }
    return {
        as "ready";
        from .answer;
    }
}
""")
wrap_graphviz(tree.toGraphViz())

## Calls

**Note**:
 - 

In [6]:
tree = frontend("""
prompt mmlu_main {
    is {
        topic is text;
        question is text;
        choices[4] is {
            value is text;
            correct is bool;
        }
        answer is repeat(.choices.value);
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .choices call {
            entry mmlu_annot;
            kwarg topic    from ?topic;
            kwarg question from ?question;
            kwarg choice   map  ?choices;
            bind choice as value;
        }
    }
    return {
        from .answer;
    }
}

prompt mmlu_annot {
    is {
        topic    is text;
        question is text;
        choice   is text;
        correct  is bool;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .choice   from ?choice;
    }
    return {
        from .choice;
        from .correct;
    }
}
""")
wrap_graphviz(tree.toGraphViz())

# MMLU Application (WIP)

Goal is to integrate the various pattern define above into a "final" MMLU appication.

In [8]:
tree = frontend("""
argument N=4;
argument main_trip_count=5;
argument annot_trip_count=3;

format thought {
    argument N=20;
    is text<$N>;
    annotate f"Intermediate thoughts used to reach reflect, at most {N} tokens.";
}

format sentence {
    argument N=50;
    is text<$N>;
    annotate f"A grammatically correct sentence made of at most {N} tokens.";
}

format correctness {
    is enum("yes","no","maybe");
    annotate "Evaluation of the correctness of a statement, can be yes, no, or maybe.";
}

struct a_choice {
    is {
        value   is text;
        correct is correctness;
        comment is sentence;
    }
    annotate {
        .value   as "the value of the choice offered to answer a question";
        .correct as "evaluation of the correctness of that choice";
        .comment as "explaination of the evaluation of this choice";
    }
}

define annot_topic="the general category from which the question was taken";
define annot_question="the question that you have to answer";
define annot_hint="if present, this is a comment you provided earlier to use as a hint";
define annot_reflect="evaluate your answer, does it seem correct?";
define annot_comment="provide a comment to be a hint in your next attempt";

prompt mmlu_main {
    is {
        topic         is text;
        question      is text;
        choices[$N]   is a_choice;
        hint[0:1]     is sentence;
        work[3:10]    is thought;
        answer        is repeat(.choices.value);
        reflect[3:10] is thought;
        comment       is sentence;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .hint     from .comment;
        to .choices call {
            entry mmlu_annot;
            kwarg topic    from ?topic;
            kwarg question from ?question;
            kwarg choice   map  ?choices;
            bind choice as value;
        }
    }
    flow {
        to mmlu_main[$main_trip_count] as "retry";
    }
    return {
        as "ready";
        from .answer;
    }
    annotate {
        .topic    as $annot_topic;
        .question as $annot_question;
        .choices  as f"the {N} choices provided with your judgement of their correctness";
        .hint     as $annot_hint;
        .work     as "think about the choices and decide which one provides the best answer to the question";
        .answer   as "you repeat the value of the choice that best answer the question";
        .reflect  as $annot_reflect;
        .comment  as $annot_comment;
    }
}

prompt mmlu_annot {
    is {
        topic         is text;
        question      is text;
        choice        is text;
        hint[0:1]     is sentence;
        work[3:10]    is thought;
        correct       is correctness;
        reflect[3:10] is thought;
        comment       is sentence;
    }
    channel {
        to .topic    from ?topic;
        to .question from ?question;
        to .choice   from ?choice;
        to .hint     from .comment;
    }
    flow {
        to mmlu_annot[$annot_trip_count] as "retry";
    }
    return {
        as "ready";
        from .choice;
        from .correct;
        from .comment;
    }
    annotate {
        .topic    as $annot_topic;
        .question as $annot_question;
        .choice   as "one of the {N} choices provided which may be the best answer to the question";
        .hint     as $annot_hint;
        .work     as "think about this choice and decide whether it could answer the question";
        .correct as "evaluation of the correctness of that choice";
        .reflect  as $annot_reflect;
        .comment  as $annot_comment;
    }
}
""")
wrap_graphviz(tree.toGraphViz())